In [1]:
# Mounting Google Drive
from google.colab import drive
from os.path import join
ROOT = '/content/drive'
print(ROOT)
drive.mount(ROOT)

/content/drive
Mounted at /content/drive


In [14]:
!pwd

/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition


In [15]:
%cd /content/drive/My\ Drive/Colab Notebooks/ProjectData

/content/drive/My Drive/Colab Notebooks/ProjectData


## Importing the dataset and necessary libraries

In [16]:
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
%matplotlib inline

In [17]:
Dataset_path = "FaceRecognition/dataset"
face_detection_path= "FaceRecognition/face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"
proto_path = "FaceRecognition/face_detection_model/deploy.prototxt"
embedding_gen_path = "FaceRecognition/openface_nn4.small2.v1.t7"
output_path = "FaceRecognition/output"

In [18]:
print("Loading face detector.....")
detector = cv2.dnn.readNetFromCaffe(proto_path, face_detection_path)

Loading face detector.....


In [19]:
print("Loading face embedder....")
embedder = cv2.dnn.readNetFromTorch(embedding_gen_path);

Loading face embedder....


In [24]:
imagepath = list(paths.list_images(Dataset_path))


In [25]:
imagepath

['FaceRecognition/dataset/arpan/IMG-20180925-WA0019.jpg',
 'FaceRecognition/dataset/arpan/IMG-20200307-WA0016.jpg',
 'FaceRecognition/dataset/arpan/IMG-20200520-WA0004.jpg',
 'FaceRecognition/dataset/arpan/IMG-20200902-WA0007.jpg',
 'FaceRecognition/dataset/arpan/IMG-20200902-WA0015.jpg',
 'FaceRecognition/dataset/arpan/IMG_20181224_121707408_HDR.jpg',
 'FaceRecognition/dataset/arpan/arpan1.jpg',
 'FaceRecognition/dataset/arpan/IMG_20190210_155413235.jpg',
 'FaceRecognition/dataset/arpan/IMG-20180811-WA0022.jpg',
 'FaceRecognition/dataset/arpan/IMG-20180208-WA0000.jpg',
 'FaceRecognition/dataset/arpan/IMG-20180104-WA0004.jpg',
 'FaceRecognition/dataset/arpan/IMG-20180114-WA0040.jpg',
 'FaceRecognition/dataset/daw/IMG_20200524_213142112.jpg',
 'FaceRecognition/dataset/daw/IMG_20200524_213302106.jpg',
 'FaceRecognition/dataset/daw/IMG_20200612_222328113_BURST000_COVER_TOP.jpg',
 'FaceRecognition/dataset/daw/IMG_20200704_100931392.jpg',
 'FaceRecognition/dataset/daw/IMG_20200722_235426989

In [26]:
knownEmbeddings = []
knownNames = []
total = 0
for (i, imageP) in enumerate(imagepath):
  print("Processing image ",i,"/",len(imagepath)-1)
  name = imageP.split(os.path.sep)[-2]

  image =  cv2.imread(imageP)
  image = imutils.resize(image, width=600)
  (h,w) = image.shape[:2]
  '''if(i % 5 == 0):
    plt.imshow(image)
    plt.show()
  '''
  blob = cv2.dnn.blobFromImage(cv2.resize(image, (300,300)),1.0, (300,300),(104.0, 177.0, 123.0), swapRB= False, crop = False)
  detector.setInput(blob)
  detections = detector.forward()

  for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2];

    if confidence < 0.5:
      continue

    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")


    cv2.rectangle(image, (startX, startY), (endX, endY), (0,0,255), 2)

    #text = "Confidence: ",confidence
    #image = cv2.putText(image,text,(startX, startY), cv2.FONT_HERSHEY_SIMPLEX , 0.45, (0,0,255),2) 
    face = image[startY:endY, startX:endX]
    (fH, fW) = face.shape[:2]

    if fH < 20 or fW < 20:
      continue

    faceBlob = cv2.dnn.blobFromImage(face, 1/255, (96,96), (0,0,0), swapRB = True, crop= False)

    embedder.setInput(faceBlob)
    vec = embedder.forward()
    knownNames.append(name)
    knownEmbeddings.append(vec.flatten())

    total = total+ 1



  cv2_imshow(image)



Output hidden; open in https://colab.research.google.com to view.

In [27]:
print("Serialising",total, "images and saving it into pickle files")
data = {"name":knownNames, "embedding":knownEmbeddings}
pickcle_path = output_path+"/data.pickle"
f = open(pickcle_path, "wb")
f.write(pickle.dumps(data))
f.close()

Serialising 48 images and saving it into pickle files


In [ ]:
image1 = cv2.imread("FaceRecognition/dataset/arpan/arpan1.jpg")
image1.shape

(3120, 4160, 3)

In [ ]:
detections[0,0,10]

array([0.        , 1.        , 0.11751852, 0.6414304 , 0.40743244,
       0.7449405 , 0.6069201 ], dtype=float32)

In [ ]:
knownNames

['arpan',
 'arpan',
 'arpan',
 'arpan',
 'arpan',
 'arpan',
 'arpan',
 'daw',
 'daw',
 'daw',
 'daw',
 'daw',
 'daw',
 'daw',
 'remon',
 'remon',
 'remon',
 'remon',
 'remon',
 'remon',
 'remon',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown']

In [ ]:
!pwd

/content/drive/My Drive/Colab Notebooks/ProjectData


In [29]:
%cd FaceRecognition/

/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition


In [30]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	deleted:    dataset/unknown/u3.jpg
	modified:   output/data.pickle

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	dataset/arpan/IMG-20180104-WA0004.jpg
	dataset/arpan/IMG-20180114-WA0040.jpg
	dataset/arpan/IMG-20180208-WA0000.jpg
	dataset/arpan/IMG-20180811-WA0022.jpg
	dataset/arpan/IMG_20190210_155413235.jpg
	dataset/daw/1579346813359.jpg
	dataset/daw/1579422263936.jpg
	dataset/daw/IMG_20190829_202315156.jpg
	dataset/daw/IMG_20191111_113042192_BURST000_COVER_TOP.jpg
	dataset/daw/IMG_20200403_222608092.jpg
	dataset/remon/IMG-20181015-WA0038.jpg
	dataset/remon/IMG-20191028-WA0035.jpg
	dataset/remon/IMG-20191028-WA0044.jpg
	dataset/remon/IMG-20191028-WA0045.jpg
	dataset/remon/IMG-20191231-WA0016.jpg
	dataset/unknown

In [31]:
!git add .


In [32]:
!git config --global user.email "suranjandaw@gmail.com"
!git config --global user.name "SuranjanDaw"

In [33]:
!git commit -m"More data added"

[master f86d358] More data added
 23 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 dataset/arpan/IMG-20180104-WA0004.jpg
 create mode 100644 dataset/arpan/IMG-20180114-WA0040.jpg
 create mode 100644 dataset/arpan/IMG-20180208-WA0000.jpg
 create mode 100644 dataset/arpan/IMG-20180811-WA0022.jpg
 create mode 100644 dataset/arpan/IMG_20190210_155413235.jpg
 create mode 100644 dataset/daw/1579346813359.jpg
 create mode 100644 dataset/daw/1579422263936.jpg
 create mode 100644 dataset/daw/IMG_20190829_202315156.jpg
 create mode 100644 dataset/daw/IMG_20191111_113042192_BURST000_COVER_TOP.jpg
 create mode 100644 dataset/daw/IMG_20200403_222608092.jpg
 create mode 100644 dataset/remon/IMG-20181015-WA0038.jpg
 create mode 100644 dataset/remon/IMG-20191028-WA0035.jpg
 create mode 100644 dataset/remon/IMG-20191028-WA0044.jpg
 create mode 100644 dataset/remon/IMG-20191028-WA0045.jpg
 create mode 100644 dataset/remon/IMG-20191231-WA0016.jpg
 create mode 100644 dataset/unknown/N

In [34]:
!git remote -v

origin	https://60b7b22abf0161d85fc0dc3684456d603fe79057@github.com/SuranjanDaw/FaceRecognition.git (fetch)
origin	https://60b7b22abf0161d85fc0dc3684456d603fe79057@github.com/SuranjanDaw/FaceRecognition.git (push)


In [35]:
!git push origin master

Counting objects: 30, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (30/30), done.
Writing objects: 100% (30/30), 2.62 MiB | 7.22 MiB/s, done.
Total 30 (delta 5), reused 0 (delta 0)
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To https://github.com/SuranjanDaw/FaceRecognition.git
   85fb770..f86d358  master -> master


In [ ]:
!git pull origin master

remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/SuranjanDaw/FaceRecognition
 * branch            master     -> FETCH_HEAD
   f848202..b7d02dc  master     -> origin/master
hint: Waiting for your editor to close the file... error: unable to start editor 'editor'
Not committing merge; use 'git commit' to complete the merge.


In [ ]:
!git status

On branch master
Your branch and 'origin/master' have diverged,
and have 1 and 1 different commits each, respectively.
  (use "git pull" to merge the remote branch into yours)

All conflicts fixed but you are still merging.
  (use "git commit" to conclude merge)

Changes to be committed:

	new file:   DataSet_setup.ipynb



In [ ]:
!git commit -m"merging pull"

[master 297ba50] merging pull


In [36]:
!git log --graph --oneline

* f86d358 (HEAD -> master, origin/master, origin/HEAD) More data added
* 85fb770 Image recognition integrated
*   297ba50 merging pull
|\  
| * b7d02dc Importing DataSet from git to google drive and setting project structure
* | 01542f3 Image detection integrated with 128D embedding generated
|/  
* f848202 output and image folder added with test image
* dfae9ee face_detection_model (caffe)added and 128-D embedder added
* b7bfb1c dataset added
* 6cbdb5c Environment Setup done
* 8b21f8c Created using Colaboratory
* 75cde1d Created using Colaboratory
* 96772a2 Create README.md


In [ ]:
!git push origin master

Counting objects: 11, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (10/10), done.
Writing objects: 100% (11/11), 263.67 KiB | 9.42 MiB/s, done.
Total 11 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 1 local object.
To https://github.com/SuranjanDaw/FaceRecognition.git
   b7d02dc..297ba50  master -> master
